# Metal Prices - merge and tidy

In [1]:
import pandas as pd

from datetime import datetime
from glob import glob

## Define helper functions

In [2]:
def read_csv_data(base_path):
    return [
        pd.read_csv(file, header=[0,1,2,3,4]) 
        for file 
        in glob(f'{base_path}/*.csv')
    ]

In [3]:
def reshape_dataframe(df):
    return (
        df
        .drop('Lease Rates', 1)
        .melt(
            id_vars=[('Market', 'Metal/Currency', 'Priced In', 'Price Type', 'Summary')], 
            var_name=['market', 'metal', 'currency', 'type','level'], 
            value_name='price'
        )
        .rename({('Market', 'Metal/Currency', 'Priced In', 'Price Type', 'Summary'): 'Date'}, axis=1)
        .dropna()
    )

## Import all files and process them

In [4]:
df_list = read_csv_data('./data')
print(f'Read {len(df_list)} files\n')

melted_dfs = [reshape_dataframe(df) for df in df_list]
for df in melted_dfs:
    print(df.head(), '\n')

Read 4 files

       Date           market metal currency type level   price
3  04/01/16  Perth Mint Spot  Plat      USD  Bid  High  878.34
4  05/01/16  Perth Mint Spot  Plat      USD  Bid  High  880.90
5  06/01/16  Perth Mint Spot  Plat      USD  Bid  High  882.10
6  07/01/16  Perth Mint Spot  Plat      USD  Bid  High  872.85
7  08/01/16  Perth Mint Spot  Plat      USD  Bid  High  868.80 

       Date           market metal currency type level   price
3  04/01/16  Perth Mint Spot  Pald      USD  Bid  High  552.00
4  05/01/16  Perth Mint Spot  Pald      USD  Bid  High  535.95
5  06/01/16  Perth Mint Spot  Pald      USD  Bid  High  526.50
6  07/01/16  Perth Mint Spot  Pald      USD  Bid  High  499.35
7  08/01/16  Perth Mint Spot  Pald      USD  Bid  High  489.50 

       Date           market metal currency type level  price
3  04/01/16  Perth Mint Spot  Silv      USD  Bid  High  13.71
4  05/01/16  Perth Mint Spot  Silv      USD  Bid  High  13.74
5  06/01/16  Perth Mint Spot  Silv      

/home/robota/propro/metal/env/lib/python3.8/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## Concat and save

In [5]:
merged_df = pd.concat(melted_dfs)
merged_df.to_csv(f'output/{datetime.utcnow().isoformat()}.csv', index=False)